In [34]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import BatchNormalization,MaxPooling2D,Conv2D,Dropout,Permute,Flatten,Dense
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler,EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,Callback

In [36]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
input_layer_shape=(48,48,1)
num_of_classes=7

In [37]:
validation_data_generator=ImageDataGenerator(rescale=1./255)

In [38]:
train_directory='/Users/lav/Downloads/fer2013/train'

In [39]:
validation_directory='/Users/lav/Downloads/fer2013/validation'

In [40]:
train_data=train_datagen.flow_from_directory(train_directory,target_size=(48,48),color_mode='grayscale',
                                            batch_size=16)

Found 28709 images belonging to 7 classes.


In [41]:
validation_data=validation_data_generator.flow_from_directory(validation_directory,target_size=(48,48),color_mode='grayscale',
                                            batch_size=16)

Found 3589 images belonging to 7 classes.


### Little VGG

In [42]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'
                 ,input_shape=input_layer_shape))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'
                 ,input_shape=input_layer_shape))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'
                 ,input_shape=input_layer_shape))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'
                 ,input_shape=input_layer_shape))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),activation='elu',padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_of_classes,activation='softmax',kernel_initializer='he_normal'))

In [43]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
batch_normalization_36 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
batch_normalization_37 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 24, 24, 64)       

In [44]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [45]:
plateau=ReduceLROnPlateau(monitor='val_loss',min_delta=0.00001)
early_stop=EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=6,
                        restore_best_weights=True)

In [46]:
callbacks=[plateau,early_stop]

In [47]:
%%time
model.fit_generator(train_data,validation_data=validation_data,callbacks=callbacks,
                   epochs=50,verbose=1)

Epoch 1/50
1795/1795 [==============================] - 328s 182ms/step - loss: 2.2717 - accuracy: 0.1845 - val_loss: 1.8143 - val_accuracy: 0.2463
Epoch 2/50
1795/1795 [==============================] - 343s 191ms/step - loss: 1.8136 - accuracy: 0.2444 - val_loss: 1.7981 - val_accuracy: 0.2452
Epoch 3/50
1795/1795 [==============================] - 461s 257ms/step - loss: 1.7926 - accuracy: 0.2546 - val_loss: 1.7530 - val_accuracy: 0.2850
Epoch 4/50
1795/1795 [==============================] - 552s 307ms/step - loss: 1.7605 - accuracy: 0.2761 - val_loss: 1.6610 - val_accuracy: 0.3330
Epoch 5/50
1795/1795 [==============================] - 640s 356ms/step - loss: 1.6941 - accuracy: 0.3099 - val_loss: 1.5658 - val_accuracy: 0.3856
Epoch 6/50
1795/1795 [==============================] - 612s 341ms/step - loss: 1.6244 - accuracy: 0.3640 - val_loss: 1.5317 - val_accuracy: 0.4188
Epoch 7/50
1795/1795 [==============================] - 763s 425ms/step - loss: 1.5606 - accuracy: 0.3951 - val_